In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../LIB/')
from env import ENV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
X = pd.read_csv(ENV.credit_card_balance_ori.value)

In [3]:
class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)
        

In [4]:
def calculate_na(ser):
    return np.sum(ser.isnull())

def view_hist(ser):
    plt.hist(ser, range=(min(ser.values), max(ser.values)))
    
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [5]:
p = scan_nan_portion(X)
p = p.sort_values()
print(p.describe())
oe = ordinal_encoder()

count    23.000000
mean      0.066541
std       0.090333
min       0.000000
25%       0.000000
50%       0.000000
75%       0.195249
max       0.199981
dtype: float64


# Export NAN

In [6]:
p.iloc[0:20]

SK_ID_PREV                    0.000000
NAME_CONTRACT_STATUS          0.000000
CNT_DRAWINGS_CURRENT          0.000000
AMT_TOTAL_RECEIVABLE          0.000000
AMT_RECIVABLE                 0.000000
AMT_RECEIVABLE_PRINCIPAL      0.000000
SK_DPD                        0.000000
AMT_PAYMENT_TOTAL_CURRENT     0.000000
AMT_DRAWINGS_CURRENT          0.000000
AMT_CREDIT_LIMIT_ACTUAL       0.000000
AMT_BALANCE                   0.000000
MONTHS_BALANCE                0.000000
SK_ID_CURR                    0.000000
SK_DPD_DEF                    0.000000
AMT_INST_MIN_REGULARITY       0.079482
CNT_INSTALMENT_MATURE_CUM     0.079482
AMT_DRAWINGS_POS_CURRENT      0.195249
AMT_DRAWINGS_ATM_CURRENT      0.195249
CNT_DRAWINGS_ATM_CURRENT      0.195249
CNT_DRAWINGS_OTHER_CURRENT    0.195249
dtype: float64

# Data Clean

## 0 - 20

In [7]:
category_20 = []
failed = []
for each in p.iloc[:].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_20.append(each)
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('========= End ==================')
print(failed)
# category_20.remove()

count    3.840312e+06
mean     1.904504e+06
std      5.364695e+05
min      1.000018e+06
25%      1.434385e+06
50%      1.897122e+06
75%      2.369328e+06
max      2.843496e+06
Name: SK_ID_PREV, dtype: float64
----
int64
value counts: 104307
NA percentage: 0.0
========= End ==================
count     3840312
unique          7
top        Active
freq      3698436
Name: NAME_CONTRACT_STATUS, dtype: object
----
object
NAME_CONTRACT_STATUS converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
value counts: 7
NA percentage: 0.0
========= End ==================
count    3.840312e+06
mean     7.031439e-01
std      3.190347e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.650000e+02
Name: CNT_DRAWINGS_CURRENT, dtype: float64
----
int64
value counts: 129
NA percentage: 0.0
========= End ==================
count    3.840312e+06
mean     5.809829e+04
std      1.059718e+05
min     -4.202502e+05
25%      0.000000e+00
50%      0.000000e+00
75%   

In [8]:
col = 'CNT_DRAWINGS_OTHER_CURRENT'
X[col].value_counts()

0.0     3077688
1.0       11354
2.0        1076
3.0         259
4.0          65
5.0          27
6.0          11
7.0           9
10.0          3
8.0           3
12.0          1
Name: CNT_DRAWINGS_OTHER_CURRENT, dtype: int64

### Ordinal Encoding

In [9]:
col = 'NAME_CONTRACT_STATUS'

oe.fit(X[col],NA_VALUE='XNA')
X[col] = oe.transform(X[col])
category_20.append(col)
###############################################



### FillNa

In [10]:
###############################################

col = 'AMT_INST_MIN_REGULARITY'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'CNT_INSTALMENT_MATURE_CUM'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'AMT_DRAWINGS_POS_CURRENT'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'AMT_DRAWINGS_ATM_CURRENT'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'CNT_DRAWINGS_ATM_CURRENT'
X[col] = X[col].fillna(int(X[col].mean()))

###############################################

col = 'CNT_DRAWINGS_OTHER_CURRENT'
X[col] = X[col].fillna(int(X[col].mean()))

###############################################

col = 'CNT_DRAWINGS_POS_CURRENT'
X[col] = X[col].fillna(int(X[col].mean()))

###############################################

col = 'AMT_DRAWINGS_OTHER_CURRENT'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'AMT_PAYMENT_CURRENT'
X[col] = X[col].fillna(X[col].mean())

# Saving

In [11]:
X.to_pickle(ENV.credit_card_balance_clean.value)

In [12]:
np.sum(X.isnull())

SK_ID_PREV                    0
SK_ID_CURR                    0
MONTHS_BALANCE                0
AMT_BALANCE                   0
AMT_CREDIT_LIMIT_ACTUAL       0
AMT_DRAWINGS_ATM_CURRENT      0
AMT_DRAWINGS_CURRENT          0
AMT_DRAWINGS_OTHER_CURRENT    0
AMT_DRAWINGS_POS_CURRENT      0
AMT_INST_MIN_REGULARITY       0
AMT_PAYMENT_CURRENT           0
AMT_PAYMENT_TOTAL_CURRENT     0
AMT_RECEIVABLE_PRINCIPAL      0
AMT_RECIVABLE                 0
AMT_TOTAL_RECEIVABLE          0
CNT_DRAWINGS_ATM_CURRENT      0
CNT_DRAWINGS_CURRENT          0
CNT_DRAWINGS_OTHER_CURRENT    0
CNT_DRAWINGS_POS_CURRENT      0
CNT_INSTALMENT_MATURE_CUM     0
NAME_CONTRACT_STATUS          0
SK_DPD                        0
SK_DPD_DEF                    0
dtype: int64

In [15]:
categori_col = list(set(category_20))

In [14]:
import pickle
categori_col_exist = pickle.load(open(ENV.clean_categorical_col.value,'rb'))
categori_col = list(set(categori_col + categori_col_exist))
pickle.dump(categori_col,open(ENV.clean_categorical_col.value,'wb'))